# EDA

In [2]:
import pandas as pd
import numpy as np
import math
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots

In [4]:
df = pd.read_csv("gen/historical_data_1d.csv", sep=';')
df["begin"] = pd.to_datetime(df["begin"])

In [5]:
df.head()

,begin,open,close,high,low,value,volume,name
0,2007-07-20,109.0,109.20,111.11,108.01,2.211623e+09,20252489,SBER
1,2007-07-23,109.7,112.00,112.65,108.10,3.901829e+09,35092029,SBER
2,2007-07-24,112.1,109.53,113.05,109.47,3.536964e+09,31762001,SBER
3,2007-07-25,108.5,109.63,109.98,107.10,4.300853e+09,39700937,SBER
4,2007-07-26,110.0,104.18,110.42,103.30,5.299123e+09,49871205,SBER


In [7]:
df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18899 entries, 0 to 18898
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   begin   18899 non-null  datetime64[ns]
 1   open    18899 non-null  float64       
 2   close   18899 non-null  float64       
 3   high    18899 non-null  float64       
 4   low     18899 non-null  float64       
 5   value   18899 non-null  float64       
 6   volume  18899 non-null  int64         
 7   name    18899 non-null  object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 1.2+ MB


In [8]:
df.isnull().sum()

,0
begin,0
open,0
close,0
high,0
low,0
value,0
volume,0
name,0


Оставим последние три дня

In [9]:
df_sber = df[df['name'] == 'SBER']
df_sber = df_sber.loc[df_sber['begin'] >= df_sber['begin'].max() - pd.Timedelta(days=252)]

In [10]:
df_sber.head()

,begin,open,close,high,low,value,volume,name
4391,2025-02-03,280.21,279.55,280.35,278.00,8.477126e+09,30373760,SBER
4392,2025-02-04,279.55,277.40,281.41,275.70,6.685825e+09,23986670,SBER
4393,2025-02-05,277.30,282.88,283.89,275.34,1.203512e+10,43070410,SBER
4394,2025-02-06,283.89,287.15,288.92,282.09,1.988558e+10,69363150,SBER
4395,2025-02-07,287.66,285.81,288.90,285.08,7.924869e+09,27626070,SBER


In [11]:
df_sber.describe()

,begin,open,close,high,low,value,volume
count,228,228.000000,228.000000,228.000000,228.000000,2.280000e+02,2.280000e+02
mean,2025-06-12 07:47:22.105263104,307.287500,307.154035,309.876798,304.447018,9.912874e+09,3.233039e+07
min,2025-02-03 00:00:00,277.300000,277.400000,280.350000,275.340000,1.329026e+08,4.160400e+05
25%,2025-04-10 18:00:00,302.667500,302.340000,305.037500,300.007500,4.148039e+09,1.393127e+07
50%,2025-06-13 12:00:00,309.995000,309.875000,311.840000,307.815000,8.402858e+09,2.758140e+07
75%,2025-08-15 06:00:00,314.540000,314.002500,317.182500,311.710000,1.350062e+10,4.259533e+07
max,2025-10-13 00:00:00,326.200000,327.050000,329.770000,323.820000,5.627715e+10,1.858167e+08
std,NaN,10.525267,10.505888,10.130239,11.066669,8.474791e+09,2.767267e+07


## plan

Остновная задача найти моменты когда стоимость актива недооценена или переоценена.
Гипотеза 1: у сбера есть большая ИТ инфраструктура, из-за этого может быть корреляция с ИТ компаниями.
Гипотеза 2: т.к. сбер - банк, то стоимость может зависеть от ставки ЦБ. Это может привести к повышенной волатильности в периоды заседания о изменении ставки.

С помощью ts выделить основные компоненты временного ряда: тред, шум, сезонность.

Выбрать индикаторы и их параметры

Посмотреть на кореляцию между разными активами. Мб сравнить сбер с бигтехом.
Выбрать тенические индикаторы для доп. анализа ряда.


In [125]:
def CreateGraph(df, plot_rows = 3, plot_columns = 1,
                row_heights = [0.75, 0.05, 0.25],
                subplot_titles = ("Price & Trends", "", "Volume")):
    # Calculate trends
    df['SMA_20'] = df['close'].rolling(window=20).mean()
    df['SMA_60'] = df['close'].rolling(window=60).mean()

    # Calculate volume spikes
    volume_mean = df['volume'].mean()
    df['volume_spike'] = df['volume'] > 2 * volume_mean # may be we can put another coefficient here

    fig = make_subplots(
        rows=plot_rows, cols=plot_columns, shared_xaxes=True,
        row_heights=row_heights,
        subplot_titles=subplot_titles
    )

    # Plot candlestick chart
    fig.add_trace(
        go.Candlestick(
            x=df['begin'],
            open=df['open'], high=df['high'],
            low=df['low'], close=df['close'],
            name='Price'
        ),
        row=1, col=1
    )

    # Plot trend lines (SMA)
    fig.add_trace(
        go.Scatter(
            x=df['begin'],
            y=df['SMA_20'],
            mode='lines',
            line=dict(color='orange', width=2),
            name='SMA 20'
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=df['begin'],
            y=df['SMA_60'],
            mode='lines',
            line=dict(color='blue', width=2),
            name='SMA 60'
        ),
        row=1, col=1
    )

    # Plot volumes
    fig.add_trace(
        go.Bar(
            x=df["begin"],
            y=df["volume"],
            marker_color=np.where(df['volume_spike'], 'crimson', 'lightblue'),
            name='Volume'
        ),
        row=3, col=1
    )

    # --------------------- #

    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)

    fig.update_layout(
        height=300 * plot_rows,
        showlegend=True,

    )

    return fig

CreateGraph(df_sber).show()

## Rolling volatility

In [13]:
def DrawVolatility(df, volatilityColumn1, volatilityColumn2 = None):
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True,
        row_heights=[0.8, 0.2], vertical_spacing=0.03,
        specs=[[{"secondary_y": True}], [{}]],
        subplot_titles=(
            f"Volatility: {volatilityColumn1}" + (f" и {volatilityColumn2}" if volatilityColumn2 else ""),
            "Volume"
        )
    )
    fig.add_trace(
        go.Candlestick(
            x=df['begin'],
            open=df['open'], high=df['high'],
            low=df['low'], close=df['close'],
            name='Price',
            increasing_line_color='rgba(0, 200, 0, 1)',
            decreasing_line_color='rgba(200, 0, 0, 1)'
        ),
        row=1, col=1,
        secondary_y=False
    )
    fig.add_trace(
        go.Bar(
            x=df["begin"], y=df["volume"],
            name="Volume",
        ),
        row=2, col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=df['begin'],
            y=df[volatilityColumn1],
            mode='lines',
            name=volatilityColumn1,
            line=dict(color='rgba(255, 0, 255, 1)')
        ),
        row=1, col=1,
        secondary_y=True
    )

    if volatilityColumn2:
        fig.add_trace(
            go.Scatter(
                x=df['begin'],
                y=df[volatilityColumn2],
                mode='lines',
                name=volatilityColumn2,
                line=dict(color='rgba(255, 255, 0, 1)')
            ),
            row=1, col=1,
            secondary_y=True
        )

    fig.update_xaxes(rangebreaks=[
        #dict(bounds=['sat', 'mon']),
        #dict(bounds=[23 + 50/60, 6 + 50/60], pattern='hour'),
    ])
    fig.update_yaxes(fixedrange=False)
    fig.update_layout(
        height=900,
        xaxis_rangeslider_visible=False
    )

    fig.show()


def RollingVolatility(df, window):
    ret = np.log(df['close']).diff()
    return ret.rolling(window=window).std() * np.sqrt(252)

def VolatilityParkinson(df, window):
    ret = (1.0 / (4.0 * math.log(2.0))) * ((df["high"] / df["low"]).apply(np.log)) ** 2.0
    return ret.rolling(window=window).apply(lambda x : (252 * x.mean()) ** 0.5)

def VolatilityGarmanKlass(df, window):
    log_hl = (df["high"] / df["low"]).apply(np.log)
    log_co = (df["close"] / df["open"]).apply(np.log)
    ret = 0.5 * log_hl ** 2 - (2 * math.log(2) - 1) * log_co ** 2
    return ret.rolling(window=window).apply(lambda x : (252 * x.mean()) ** 0.5)

df_sber['RollingVolatility_7d'] = RollingVolatility(df_sber, 7)
df_sber['RollingVolatility_30d'] = RollingVolatility(df_sber, 30)

df_sber['VolatilityParkinson_7d'] = VolatilityParkinson(df_sber, 7)
df_sber['VolatilityParkinson_30d'] = VolatilityParkinson(df_sber, 30)

df_sber['VolatilityGarmanKlass_7d'] = VolatilityGarmanKlass(df_sber, 7)
df_sber['VolatilityGarmanKlass_30d'] = VolatilityGarmanKlass(df_sber, 30)

#print(df_sber)


DrawVolatility(df_sber, 'RollingVolatility_7d', 'RollingVolatility_30d')
DrawVolatility(df_sber, 'VolatilityParkinson_7d', 'VolatilityParkinson_30d')
DrawVolatility(df_sber, 'VolatilityGarmanKlass_7d', 'VolatilityGarmanKlass_30d')

# Анализ ACF/PACF для 30-дневной волатильности

## Анализ ACF (автокорреляционная функция)
- **Медленное затухание**: ACF показывает очень медленное, почти линейное затухание, что характерно для нестационарных процессов.
- **Высокая персистентность**: Значимые автокорреляции наблюдаются на многих лагах, что указывает на сильную зависимость текущих значений волатильности от предыдущих.
- **Признак длинной памяти**: Такое поведение типично для финансовых временных рядов волатильности.

## Анализ PACF (частичная автокорреляционная функция)
- **Резкий обрыв после первого лага**: PACF показывает значимую корреляцию только на лаге 1, затем резко обрывается.
- Это указывает на возможный процесс AR(1) для моделирования волатильности.

## Основные выводы
- **Нестационарность ряда**: Требуется дифференцирование для стационаризации.
- **ARIMA-моделирование**: Подходит модель типа ARIMA(p,d,q) где:
  - d ≥ 1 (требуется дифференцирование)
  - p = 1 (на основе PACF)
  - q может быть небольшим (на основе ACF)
- **Рекомендуемая модель**: ARIMA(1,1,0) или ARIMA(1,1,1) могут быть хорошей отправной точкой.
- **Альтернативные подходы**: Также стоит рассмотреть модели GARCH для прямого моделирования волатильности, учитывая характер финансовых данных.

In [37]:
from statsmodels.tsa.stattools import acf, pacf

volatility = df_sber['RollingVolatility_30d'].dropna()

acf_vals = acf(volatility, nlags=50)
pacf_vals = pacf(volatility, nlags=50)

lags = list(range(len(acf_vals)))

fig = make_subplots(rows=2, cols=1, subplot_titles=("ACF", "PACF"))

fig.add_trace(go.Bar(x=lags, y=acf_vals, name='ACF'), row=1, col=1)
fig.add_trace(go.Bar(x=lags, y=pacf_vals, name='PACF'), row=2, col=1)

fig.update_layout(height=600, title="ACF and PACF of 30d rolling volatility")
fig.show()

In [17]:
volatilities = pd.DataFrame({
    "roling": df_sber['RollingVolatility_7d'],
    "parkinson": df_sber['VolatilityParkinson_7d'],
    "garmanklass": df_sber['VolatilityGarmanKlass_7d']
})
corr_pearson = volatilities.corr(method='pearson')
corr_spearman = volatilities.corr(method='spearman')

print(corr_pearson)
fig = px.imshow(
    corr_pearson,
    aspect='auto'
)
fig.show()

print(corr_spearman)
fig = px.imshow(
    corr_spearman,
    aspect='auto'
)
fig.show()


               roling  parkinson  garmanklass
roling       1.000000   0.869840     0.798737
parkinson    0.869840   1.000000     0.986781
garmanklass  0.798737   0.986781     1.000000


               roling  parkinson  garmanklass
roling       1.000000   0.863139     0.785274
parkinson    0.863139   1.000000     0.982376
garmanklass  0.785274   0.982376     1.000000


## Returns


In [21]:
def IntradayReturn(df):
    return df['close']/df['open'] - 1

def OvernightReturn(df):
    return df['open']/df['close'].shift(1) - 1

def Amplitude(df):
    return (df['high'] - df['low']) / ((df['high'] + df['low']) / 2)

def OHLC(df):
    return (df['open'] + df['close'] + df['low'] + df['high'])/4.0

def MedianPrice(df):
    return (df['low'] + df['high'])/2.0

def BodyMedian(df):
    return (df['open'] + df['close'])/2.0

df_sber['intraday'] = IntradayReturn(df_sber)
df_sber['overnight'] = OvernightReturn(df_sber)
df_sber['amplitude'] = Amplitude(df_sber)

df_sber['ohlc'] = OHLC(df_sber)
df_sber['MedianPrice'] = MedianPrice(df_sber)
df_sber['BodyMedian'] = BodyMedian(df_sber)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_sber['begin'],
    y=df_sber['intraday'],
    mode='lines',
    name='intraday'
))
fig.add_trace(go.Scatter(
    x=df_sber['begin'],
    y=df_sber['overnight'],
    mode='lines',
    name='overnight'
))
fig.add_trace(go.Scatter(
    x=df_sber['begin'],
    y=df_sber['amplitude'],
    mode='lines',
    name='amplitude'
))


fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_sber['begin'],
    y=df_sber['ohlc'],
    mode='lines',
    name='ohlc'
))
fig.add_trace(go.Scatter(
    x=df_sber['begin'],
    y=df_sber['MedianPrice'],
    mode='lines',
    name='MedianPrice'
))
fig.add_trace(go.Scatter(
    x=df_sber['begin'],
    y=df_sber['BodyMedian'],
    mode='lines',
    name='BodyMedian'
))
fig.show()


In [29]:
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=False,
    vertical_spacing=0.08,
    subplot_titles=("Intraday Returns", "Overnight Returns")
)

# Plot overnight
fig.add_trace(
    go.Histogram(x=df_sber['intraday'], nbinsx=100, name='Intraday', marker_color='blue'),
    row=1, col=1
)

# Overnight
fig.add_trace(
    go.Histogram(x=df_sber['overnight'], nbinsx=100, name='Overnight', marker_color='green'),
    row=2, col=1
)

fig.update_layout(
    height=600,
    showlegend=False,
    title_text="Distributions"
)

fig.show()

In [30]:
df_close = df.pivot(index='begin', columns='name', values='close')

returns = df_close.pct_change(fill_method=None)

# correlation relative to returns
corr_tickers = returns.corr()
annot_text = [[f"{val:.3f}" for val in row] for row in corr_tickers.values]

fig = ff.create_annotated_heatmap(
    z=corr_tickers.values,
    x=corr_tickers.columns.tolist(),
    y=corr_tickers.index.tolist(),
    annotation_text=annot_text,
    showscale=True
)
fig.update_layout(title='Correlation matrix of daily returns by tickers')
fig.show()

In [146]:
import seaborn as sns
import matplotlib.pyplot as plt

corr_values = df.groupby("name")[['close', 'volume']].apply(lambda x: x['close'].corr(x['volume']))
corrs = pd.DataFrame({'name': corr_values.index, 'price_volume_corr': corr_values.values})

display(corrs)

,name,price_volume_corr
0,CBOM,0.212973
1,MBNK,0.243527
2,SBER,-0.491790
3,SBERP,-0.372020
4,SVCB,0.049331
5,T,-0.050378
6,VTBR,-0.182982


In [120]:
def RSI(df, period = 14):
    delta = df.diff()

    up = delta.where(delta > 0, 0.0)
    down = -delta.where(delta < 0, 0.0)

    roll_up = pd.Series(up).ewm(alpha=1 / period, adjust=False).mean()
    roll_down = pd.Series(down).ewm(alpha=1 / period, adjust=False).mean()

    return 100.0 - (100.0 / (1.0 + roll_up / roll_down))

def MACD(df, fast = 12, slow = 26, signal = 9):
    ema_fast = df.ewm(span=fast, adjust=False).mean()
    ema_slow = df.ewm(span=slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    macd_signal = macd.ewm(span=signal, adjust=False).mean()
    macd_hist = macd - macd_signal
    return macd, macd_signal, macd_hist

df_sber['RSI_14'] = RSI(df_sber['close'], period=14)
df_sber['MACD'], df_sber['MACD_signal'], df_sber['MACD_hist'] = MACD(df_sber['close'])

fig = CreateGraph(df_sber, plot_rows = 5, plot_columns = 1, row_heights = [0.7, 0.05, 0.25, 0.25, 0.25], subplot_titles = ('Price', '','Volume', 'MACD', 'RSI (14)'))

fig.add_trace(go.Scatter(x=df_sber['begin'], y=df_sber['RSI_14'], name='RSI (14)', line=dict(color='blue')), row=5, col=1)
fig.add_hline(y=70, line=dict(color='red', dash='dash'), row=5, col=1)
fig.add_hline(y=30, line=dict(color='green', dash='dash'), row=5, col=1)

fig.add_trace(go.Scatter(x=df_sber['begin'], y=df_sber['MACD'], name='MACD', line=dict(color='orange')), row=4, col=1)
fig.add_trace(go.Scatter(x=df_sber['begin'], y=df_sber['MACD_signal'], name='Signal', line=dict(color='blue')), row=4, col=1)
fig.add_trace(go.Bar(x=df_sber['begin'], y=df_sber['MACD_hist'], name='Histogram', marker_color='grey'), row=4, col=1)

fig.show()

In [158]:
df_divs = pd.read_csv('gen/dividends_data.csv', sep=';')

df_divs.head()

,date,name,isin,value,currencyid
0,2019-06-13,SBER,RU0009029540,16.0,RUB
1,2020-10-05,SBER,RU0009029540,18.7,RUB
2,2021-05-12,SBER,RU0009029540,18.7,RUB
3,2023-05-11,SBER,RU0009029540,25.0,RUB
4,2024-07-11,SBER,RU0009029540,33.3,RUB


In [184]:
df_divs_sber = df_divs[(df_divs['name'] == 'SBER') &
                       (pd.to_datetime(df_divs['date']) >= df_sber['begin'].max() - pd.Timedelta(days=252))]

fig = CreateGraph(df_sber, plot_rows = 3, plot_columns = 1, row_heights = [0.7, 0.05, 0.25], subplot_titles = ('Price', '','Volume'))

for _, dividend in df_divs_sber.iterrows():
    fig.add_vrect(
        x0=pd.to_datetime(dividend['date']) - pd.Timedelta(days=1),
        x1=pd.to_datetime(dividend['date']) + pd.Timedelta(days=1),
        fillcolor="blue",
        opacity=0.2,
        line_width=1,
        layer="below",
        annotation_text=f"Dividend = {dividend['value']}{dividend['currencyid']}",
        annotation_position="outside top right",
        row=1, col=1
    )

fig.show()